In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

#### Sử dụng vector hóa tf-idf
    -Mạnh về khoảng cách các từ
    - SVD giảm chiều dữ liệu về  75
    -chuẩn hóa dữ liệu sử dụng MinMax
    -Model : 
           *Logistic Regression
           *SVM với hệ số điều chỉnh độ rộng của lề C=1
    -Độ đo sử dụng trong các mô hình là ROC_AUC
    

In [3]:
class Gender:
    def __init__(self, X, y):
        self.X = X
        self.y = y

        self.tf_idf = TfidfVectorizer(stop_words=None, tokenizer=None, encoding='utf-8', lowercase=True)
        x = self.tf_idf.fit_transform(X)
        self.svd = TruncatedSVD(n_components=100, random_state=42)
        self.svd.fit(x)

        self.scale = {'mimax': MinMaxScaler(), 'stas': StandardScaler()}

        # model fit data
        self.model = {'logistic': LogisticRegression(random_state=42), 'svm': LinearSVC(C=1, random_state=42)}

    def fit(self, model_name='logistic'):
        self.model[model_name].fit(self.X, self.y)

    # vector hóa dữ liệu text X và chuẩn hóa nó
    def vertorizer(self, X, scale_name='mimax'):
        res = self.tf_idf.transform(X)
        res = self.svd.transform(res)
        return self.scale[scale_name].fit_transform(res)

    def evaluation(self, X, y, model_name='logistic'):
        y_predict = self.model[model_name].predict(X)
        tpt, fpt, thres = roc_curve(y, y_predict)
        print(model_name, ' ROC_AUC : %.2f %'%(roc_auc_score(y, y_predict) * 100))
        plt.plot(tpt, fpt)

    def show(self):
        plt.plot([0, 1], [1 , 0])
        plt.show()

#### ĐỌC dữ liệu và sử dụng validation đánh giá mô hình qua độ đo ROC_AUC

In [4]:
path_male = 'Data/data_gender/data_male.tsv'
path_female = 'Data/data_gender/data_female.tsv'

data_male = pd.read_csv(path_male, sep='\t').to_numpy()[:, 0]
y_male = np.ones(np.shape(data_male)[0], dtype=int)

data_female = pd.read_csv(path_female, sep='\t').to_numpy()[:, 0]
y_female = -np.ones(np.shape(data_female)[0], dtype=int)

X = np.array([str(x).lower() for x in np.concatenate((data_male, data_female))])
y = np.concatenate((y_male, y_female))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

#### Thực hiện đánh giá mô hình logistic và svm

In [ ]:
gender = Gender(X_train, y_train)

gender.X = gender.vertorizer(X_train)
X_test_vectorizer = gender.vertorizer(X_test)

# Logistic
gender.fit(model_name='logistic')
print('Train :', end='')
gender.evaluation(gender.X, gender.y, model_name='logistic')
print('Test : ', end='')
gender.evaluation(X_test_vectorizer, y_test, model_name='logistic')

# SVM
gender.fit(model_name='svm')
print('Train :', end='')
gender.evaluation(gender.X, gender.y, model_name='svm')
print('Test : ', end='')
gender.evaluation(X_test_vectorizer, y_test, model_name='svm')

gender.show()

### Kết quả được thực hiện trên google colab
    Train :logistic  ROC_AUC : 73.87 %
    Test : logistic  ROC_AUC : 73.43 %
    
    Train :svm  ROC_AUC : 73.43 %
    Test : svm  ROC_AUC : 73.15 %
